In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import signal
from scipy.signal import welch

import auxlib

In [ ]:
saving_figs = os.path.join(os.getcwd(), 'figures')
if not os.path.exists(saving_figs):
	os.makedirs(saving_figs)

saving_files = os.path.join(os.getcwd(), 'media')
if not os.path.exists(saving_files):
	os.makedirs(saving_files)

In [ ]:
signals, signal_headers, header = auxlib.loadEEG(subject=1, record=1)

In [ ]:
subject, data, labels, channels = auxlib.GetSignal(1, 'RL', sample_rate=160, segment_length=640, selected_channels=None)

fs = 160.0
fig, axs = plt.subplots(16, 4, figsize=(20, 40))

signals = data[0]

for ch in range(1, 65):
	x = signals[ch-1]
	f, Pxx = welch(x, fs, scaling='density')

	Pxx_dB = 10 * np.log10(Pxx)

	axs[(ch-1)//4, (ch-1)%4].plot(f, Pxx_dB)
	axs[(ch-1)//4, (ch-1)%4].set_ylabel('PDS [dB]')
	#axs[(ch-1)//4, (ch-1)%4].set_ylim(0, 7)
	axs[(ch-1)//4, (ch-1)%4].set_xlabel('Frequency [Hz]')
	axs[(ch-1)//4, (ch-1)%4].set_title("Ch {}".format(ch))

# Layout so plots do not overlap
fig.tight_layout()
#fig.savefig('S055R03_spectrogram.png', dpi=300)
fig.savefig(os.path.join(saving_figs, 'S001R03_psd_all_channels.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
subject, data, labels, channels = auxlib.GetSignal(12, 'RR', sample_rate=160, segment_length=640, selected_channels=None)

fs = 160.0
fig, axs = plt.subplots(16, 4, figsize=(20, 40))

signals = data[0]

for ch in range(1, 65):
	x = signals[ch-1]
	f, Pxx = welch(x, fs, scaling='density')

	Pxx_dB = 10 * np.log10(Pxx)

	axs[(ch-1)//4, (ch-1)%4].plot(f, Pxx/max(Pxx))
	axs[(ch-1)//4, (ch-1)%4].set_ylabel('PDS (Normalized)')
	#axs[(ch-1)//4, (ch-1)%4].set_ylim(0, 7)
	axs[(ch-1)//4, (ch-1)%4].set_xlabel('Frequency [Hz]')
	axs[(ch-1)//4, (ch-1)%4].set_title("Ch {}".format(ch))

# Layout so plots do not overlap
fig.tight_layout()
#fig.savefig('S055R03_spectrogram.png', dpi=300)

plt.show()